# Time Series GIF from GEO Example using TileDB

This example shows how the Capella API can be used to fetch a time series stack of data, read data for a given bounding box directly from TileDB arrays, and create a time series gif for visualization. To run this notebook, you will need a Capella API account, with credentials saved in a credentials.json file.

In [ ]:
import capella
from capella import lee_filter

import json

from IPython.display import HTML
from matplotlib import animation
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import rasterio
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
from rasterio.windows import Window
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage import exposure

### Set up project variables

In [ ]:
with open('filter-gif.json') as f:
    filters = json.load(f)
    bbox = filters['bbox']

# Windows sizes for filtering
FILTSIZE = 3 # window size for speckle filter

### Use the API to search for Capella SAR data

In [ ]:
result = ! rio capella --credentials credentials.json --area filter-gif.json --collection rotterdam-aerial-mosaic --limit 50 query
fc = json.loads(result[0])
features = fc['features']

### Build a time series animation from the time series

Ingests the stack of images ordered from the API and assembles a time series animation

In [ ]:
# sort the features by datetime and create a timeseries stack
timeseries = []
features =  sorted(features, key = lambda f: f['properties']['datetime'])

rcParams['figure.figsize'] = 10, 5

fig = plt.figure("Time Series")
ax = fig.add_subplot(111)

for idx, f in enumerate(features):
    with rasterio.open(f"tiledb://capellaspace/{f['id']}") as src:
        meta = src.meta
        native_bounds = transform_bounds(CRS.from_epsg(4326), src.crs, *bbox)        
        bounds_window = src.window(*native_bounds)
        bounds_window = bounds_window.intersection(Window(0, 0, src.width, src.height))

        img = src.read(1, window=bounds_window)

        img[img == meta['nodata']] = 0
        img = lee_filter(img, FILTSIZE)
        img = exposure.adjust_log(img, gain=10)

        frame =  ax.imshow(img, cmap='gray')   
        t = ax.annotate(f["properties"]["datetime"], (10, 50), color='red', fontsize=15) # add text
        timeseries.append([frame, t])

anim = animation.ArtistAnimation(fig, timeseries, interval=350, blit=True, repeat_delay=350)
plt.close()
HTML(anim.to_html5_video())

In [ ]:
anim.save('animation.gif', writer='imagemagick', fps=5)